# checks
- robust to mean vs median
- how many trials does it fail on
- reproducible with trial averaged taus
- does age/acc also correlate with pre or encoding taus

In [4]:
%load_ext autoreload
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # high res plotting

import hdf5storage # you need to pip install this to handle matlab > 7.3 files or something

import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp, pearsonr, spearmanr
import statsmodels.formula.api as smf

from fooof import FOOOFGroup, fit_fooof_group_3d

# plotting stuff
import matplotlib.pyplot as plt
from seaborn import despine

# plot settings
font = {'family' : 'Bitstream Vera Sans',
        'weight' : 'regular',
        'size'   : 13}
#figure = {'figsize' : (16,8)}

plt.rc('font', **font)
#plt.rc('figure', **figure)

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# simple hamming + fft
def get_power(data, fs):
    data = data * np.hamming(len(data))
    p = np.abs(np.fft.rfft(data))**2
    f = np.linspace(0, fs/2, len(p))
    return f, p

def index_of_nearest_timepoint(list_of_times, timepoint):
    time_idx = np.where(list_of_times>=timepoint)
    idx = time_idx[0][0]
    return idx

# gao's knee-to-tau conversion
def convert_knee_val(knee, exponent=2.):
    """
    Convert knee parameter to frequency and time-constant value.
    Can operate on array or float.

    Default exponent value of 2 means take the square-root, but simulation shows
    taking the exp-th root returns a more accurate drop-off frequency estimate
    when the PSD is actually Lorentzian.
    """
    knee_freq = knee**(1./exponent)
    knee_tau = 1./(2*np.pi*knee_freq)
    return knee_freq, knee_tau

In [120]:
# set up all the settings: analyze only OFC/PFC data

# list of data directories, one for each subject
dirs = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10']

# these are the pfc+ofc channels for each subject
pfc = [['RCD8','RCD9','RCD10','ROF6','ROF7','ROF8','ROF9','ROF10','ROF1','ROF2','ROF3','ROF4','ROF5'],
       ['LIN10','RC8','LIN14','RC9','LIN15','RC10','LIN16','RC11','RC12','ROF11','ROF12','ROF13','ROF14','ROF15','ROF16','ROF2','ROF3','ROF4'],
       ['LOF5','ROF7','LOF6','ROF8', 'LOF2'],
       ['LCI5','ROF7','LCI6','ROF8','LCI7','RCI5','LCI8','RCI6','LCI9','RCI7','RCI8','RCI9','ROF1','ROF2'],
       ['LTG17','LTG25','LTG26','LTG33','LTG34','LTG35','LTG41','LTG42','LTG43','LTG44','OFG1','OFG2','OFG3','OFG4','OFG6','OFG7','OFG8','OFG9','OFG11','OFG12','OFG13','OFG14','OFG16','OFG17','OFG18','OFG19'],
       ['PF2','PF3','PF4','PF5','PF6','FG4','FG5','FG6','FG7','FG11','FG12','FG13','FG14','FG15','FG17','FG18','FG19','FG20','FG21','FG22','FG23','FG25','FG26','FG27','FG28','FG29','FG30','FG31','AOF1','AOF2','AOF3','AOF4','AOF5','MOF1','MOF2','MOF3','MOF4','MOF5','POF1','POF2','POF3','POF4','POF5','FG2','FG3','FG9','FG10'],
       ['LOF6','ROF9','LOF7','RAC9','LOF8','RAC10','LOF9','RIN7','LIN3','RIN8','LIN4','RIN9','LIN5','LIN6','LIN7','LIN8','LIN9','LOF2','ROF1','ROF6','ROF7','ROF8'],
       ['FOPL_04','FOPL_05','FOAL_04','FOAL_05'],
       ['FOLA_09','FOLA_10','FOLA_11','FOLA_12','FOLP_12','FOLP_13','FORA_09','FORA_10','FORA_11','FORA_12','FORP_13','FAR_02','FAR_03','FAR_04','FAR_05','FMR_04','FMR_05','FMR_06','FAL_01','FAL_02','FAL_03','FAL_04'],
       ['AVF1','AVF2','AVF3','AVF4','AVF5','AVF8','AVF9','MVF1','MVF2','MVF3','MVF4','PVF2','PVF3','PVF4','OF4','OF5','OF6','FG1','FG2','FG3','FG4','FG5','FG9','FG10','FG11','FG13','FG14','FG15','FG16','FG17','FG19','FG22','FG23','FG25','FG26','FG27','FG28','FG29','FG40','FG41','FP2','FP3','FP4','FP5','FP6','OF2','OF3']]
subs_fs = [1000, 1000, 1000, 1000, 1000, 1000, 1000, 512, 512, 1000]

# there are the three datatypes in the mat file
data_types = ['data_pre', 'data_encmain', 'data_proc']
n_types = np.shape(data_types)[0]

# these are the time windows Johnson et al. used
    # note that I'm keep all the windows 900ms so the knee-fits are all using the same amount of data
    # but we will average taus from the two delay periods since we just want to check delay versus pre-stim
time_wins = [[-0.9, 0.0], [0.6, 1.5], [0.3, 1.2]]

# fooof settings
freq_range = [2, 80] # this is the range gao uses
bw_lims = [2, 8]
max_n_peaks = 2 # single-trial fits are so noisy so this isn't super critical

# behavioral variables
ages = [33, 50, 69, 31, 22, 31, 34, 27, 34, 42]
acc = [0.9, 0.967, 0.817, 0.933, 0.792, 0.95, 0.958, 0.95, 0.842, 0.925]
rt = [1783.1, 1138.9, 1720.9, 1790.8, 2950.7, 1245.7, 722.4, 1165.2, 1602.1, 780.8]

### Epoch data and compute power spectrums

In [8]:
t_diff = []
n_freqs = 100
f_axis_all, PS_all = [], []
for i_sub in np.arange(len(dirs)):
    print(dirs[i_sub], end='|')
    filepath = '/Users/rdgao/Documents/data/CRCNS/Johnson/' + dirs[i_sub] + '/data_derived.mat'
    mat = hdf5storage.loadmat(filepath)
    fs = subs_fs[i_sub] # subject-specific sampling rate

    chans = np.squeeze(mat[data_types[0]]['label'].tolist()).tolist() # full list of channels
    n_chans = np.shape(pfc[i_sub])[0] # number of PFC+OFC channels
    chans_idx = []
    for i in np.arange(n_chans):
        pfc_chan = chans.index(pfc[i_sub][i])
        chans_idx.append(pfc_chan)


    fs = subs_fs[i_sub] # subject-specific sampling rate

    chans = np.squeeze(mat[data_types[0]]['label'].tolist()).tolist() # full list of channels
    n_chans = np.shape(pfc[i_sub])[0] # number of PFC+OFC channels

    # find indices of PFC+OFC channels in full channel list
    chans_idx = []
    for i in np.arange(n_chans):
        pfc_chan = chans.index(pfc[i_sub][i])
        chans_idx.append(pfc_chan)

    # get the number of trials, for initializing data arrays
    data = mat[data_types[0]]['trial']
    n_trials = np.shape(np.squeeze(data))[0]

    # condition window * channels * trials
    aexp = np.zeros((n_types, n_chans, n_trials)) # exponent
    aexp.fill(np.nan)
    akne = np.zeros((n_types, n_chans, n_trials)) # knee
    akne.fill(np.nan)
    fkne = np.zeros((n_types, n_chans, n_trials)) # knee freq
    fkne.fill(np.nan)
    tkne = np.zeros((n_types, n_chans, n_trials)) # tau
    tkne.fill(np.nan)

    PS_subj = np.zeros((n_types, n_chans, n_trials, n_freqs))
    for i_cond in np.arange(n_types):
        # restrict to proper time windows for this condition
        idx = [0, 0]
        times = np.asarray(np.squeeze(mat[data_types[0]]['time'].tolist()).tolist()[0])
        timepoint = time_wins[i_cond][0]
        idx[0] = index_of_nearest_timepoint(times, timepoint)
        timepoint = time_wins[i_cond][1]
        idx[1] = index_of_nearest_timepoint(times, timepoint)
        n_points = np.abs(np.diff(idx))[0]

        # condition data
        cond_data = mat[data_types[i_cond]]['trial']

        # get data array: channels by trials by times
        trial = np.zeros((n_chans, n_trials, n_points))
        for i in np.arange(n_trials):
            trial_dat = np.squeeze(np.squeeze(cond_data)).tolist()[i]
            trial[:, i, :] = trial_dat[chans_idx, idx[0]:idx[1]]

        # get power spectrum for each trial and channel
        for i_chan in np.arange(n_chans):
            for i_trial in np.arange(n_trials):
                trial_data = trial[i_chan, i_trial, :] # data vector
                freqs, PS = get_power(trial_data, fs) # power spectrum
                PS_subj[i_cond, i_chan, i_trial,:] = PS[:n_freqs]

    f_axis_all.append(freqs[:n_freqs])
    PS_all.append(PS_subj)

s1|s2|s3|s4|s5|s6|s7|s8|s9|s10|

### FOOOFing trial averaged data

In [154]:
tt_all = []
tau_avg_all = []
for i_subj in range(len(PS_all)):
    # median of all PS over trials
    PSD = np.median(PS_all[i_subj],2)
    fm = FOOOFGroup(peak_width_limits=bw_lims, aperiodic_mode='knee', max_n_peaks=max_n_peaks, verbose=False)
    fgs = fit_fooof_group_3d(fm, freqs[:n_freqs], PSD, freq_range)
    ap_params = np.array([fg.get_params('aperiodic_params')[:,1:] for fg in fgs])
    tau = convert_knee_val(ap_params[:,:,0],ap_params[:,:,1])[1]
    tau_pre = tau[0,:]
    tau_enc = (tau[1,:]+tau[2,:])/2
    tau_avg_all.append(np.nanmedian(np.array((tau_pre, tau_enc, tau_enc-tau_pre)), axis=1))
    tt_all.append(ttest_1samp(tau_enc-tau_pre,0,nan_policy='omit'))
    
tt_all = np.array(tt_all)
tau_avg_all = np.array(tau_avg_all)

In [156]:
# t-test for if electrode-level differences within subjects are all positive
print('t-stat=%.3f, p-value=%.3f'%ttest_1samp(tt_all[:,0],0,nan_policy='omit'))


t-stat=3.112, p-value=0.014


### FOOOF single trial data

In [163]:
for i_cond in range(len(data_types)):    
    fm = FOOOFGroup(peak_width_limits=bw_lims, aperiodic_mode='knee', max_n_peaks=max_n_peaks, verbose=False)
    fgs = fit_fooof_group_3d(fm, freqs[:n_freqs], PS_all[i_subj][i_cond], freq_range)


KeyboardInterrupt: 

In [164]:
fgs = fit_fooof_group_3d(fm, freqs[:n_freqs], PS_all[i_subj][i_cond], freq_range)

In [165]:
fgs[0]

In [ ]:
#     PSD = np.median(PS_all,2)
#     fm = FOOOFGroup(peak_width_limits=bw_lims, aperiodic_mode='knee', max_n_peaks=max_n_peaks)
#     fgs = fit_fooof_group_3d(fm, freqs[:n_freqs], PSD, freq_range)
#     taus_chan = np.array([convert_knee_val(fg.get_params('aperiodic_params', 'knee'), fg.get_params('aperiodic_params', 'exponent'))[1] for fg in fgs])
#     tau_pre = taus_chan[0,:]
#     tau_enc = (taus_chan[1,:]+taus_chan[2,:])/2
#     t_diff.append(tau_enc-tau_pre)

#     print(ttest_1samp(tau_enc-tau_pre, 0))


In [114]:
tp = np.array([ttest_1samp(td,0,nan_policy='omit') for td in t_diff])
ttest_1samp(tp[:,0],0, nan_policy='omit')

/Users/rdgao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Warning: converting a masked element to nan.
  """Entry point for launching an IPython kernel.


Ttest_1sampResult(statistic=3.023460412205026, pvalue=0.01647249911624109)

In [41]:
cond_data[0][0][0].shape

(33, 4501)

In [95]:
t_diff = []
n_freqs = 100
PS_all = np.zeros((n_types, n_chans, n_trials, n_freqs))
# loop over the three data time windows
for i_cond in np.arange(n_types):

    # restrict to proper time windows for this condition
    idx = [0, 0]
    times = np.asarray(np.squeeze(mat[data_types[0]]['time'].tolist()).tolist()[0])
    timepoint = time_wins[i_cond][0]
    idx[0] = index_of_nearest_timepoint(times, timepoint)
    timepoint = time_wins[i_cond][1]
    idx[1] = index_of_nearest_timepoint(times, timepoint)
    n_points = np.abs(np.diff(idx))[0]

    # condition data
    cond_data = mat[data_types[i_cond]]['trial']

    # get data array: channels by trials by times
    trial = np.zeros((n_chans, n_trials, n_points))
    for i in np.arange(n_trials):
        trial_dat = np.squeeze(np.squeeze(cond_data)).tolist()[i]
        trial[:, i, :] = trial_dat[chans_idx, idx[0]:idx[1]]

    # get knees for each trial and channel
    for i_chan in np.arange(n_chans):
        for i_trial in np.arange(n_trials):
            trial_data = trial[i_chan, i_trial, :] # data vector
            freqs, PS = get_power(trial_data, fs) # power spectrum
            PS_all[i_cond, i_chan, i_trial,:] = PS[:n_freqs]
    
PSD = np.median(PS_all,2)
fm = FOOOFGroup(peak_width_limits=bw_lims, aperiodic_mode='knee', max_n_peaks=max_n_peaks)
fgs = fit_fooof_group_3d(fm, freqs[:n_freqs], PSD, freq_range)
taus_chan = np.array([convert_knee_val(fg.get_params('aperiodic_params', 'knee'), fg.get_params('aperiodic_params', 'exponent'))[1] for fg in fgs])
tau_pre = taus_chan[0,:]
tau_enc = (taus_chan[1,:]+taus_chan[2,:])/2
t_diff.append(tau_enc-tau_pre)

ttest_1samp(tau_enc-tau_pre, 0)

Running FOOOFGroup across 13 power spectra.
Running FOOOFGroup across 13 power spectra.
Running FOOOFGroup across 13 power spectra.


Ttest_1sampResult(statistic=0.9292441887522229, pvalue=0.3710782333594197)